In [11]:
import  pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

In [12]:
sales_points = pd.read_csv("D:\\Competitions\\DataStorm-2022\\data\\SalesPointsV2.csv")

In [13]:
sales_points

,Unnamed: 0,ItemCode,#sales,stage,WeeklySales
0,0,3418,21,validation,"[29, 42, 41, 41, 44, 46, 43, 49, 54, 69, 101, ..."
1,1,3427,21,validation,"[11, 40, 20, 48, 54, 28, 36, 52, 42, 57, 46, 4..."
2,2,7666,21,validation,"[84, 119, 196, 185, 94, 15, 10, 30, 79, 82, 14..."
3,3,9925,21,test,"[1, 9, 11, 13, 5, 11, 11, 13, 12, 10, 21, 15, ..."
4,4,16936,21,test,"[9, 25, 40, 31, 21, 43, 7, 34, 151, 32, 28, 25..."
...,...,...,...,...,...
189,189,1101769,21,test,"[8, 3, 0, 0, 18, 20, 22, 17, 14, 1, 0, 2, 0, 0..."
190,190,1103056,19,test,"[8, 12, 28, 27, 20, 17, 24, 19, 12, 7, 18, 11,..."
191,191,1105009,18,validation,"[5, 5, 10, 17, 17, 4, 6, 9, 4, 7, 2, 6, 6, 7, ..."
192,192,1105018,17,test,"[3, 6, 11, 4, 7, 8, 6, 7, 10, 12, 4, 10, 10, 1..."


In [14]:
def build_arima_model(lis_data,result,order=(5,1,0),show_fig=False,show_summary=False):
    model = ARIMA(lis_data, order=order)
    model_fit = model.fit()
    if show_summary:
        # summary of fit model
        print(model_fit.summary())

    if show_fig:
        # line plot of residuals
        residuals = pd.DataFrame(model_fit.resid)
        residuals.plot()
        plt.show()
        # density plot of residuals
        residuals.plot(kind='kde')
        plt.show()
        # summary stats of residuals
        print(residuals.describe())

    result["arima_model"] = model_fit
    return result

In [15]:
selected_data_points = sales_points

In [16]:
models_list = {}
for i in tqdm(selected_data_points.index):
    result = dict()
    product_series = sales_points.iloc[i]
    result["input_data"] = product_series
    __data = eval(product_series["WeeklySales"])
    build_arima_model(__data,result,show_fig=False,show_summary=False)
    model = result["arima_model"]
    fc = model.forecast(4,alpha=0.05)
    fc_series = pd.Series(fc)
    result["week_prediction"] = fc_series
    models_list[product_series["ItemCode"]] = result

  0%|          | 0/194 [00:00<?, ?it/s]d:\competitions\datastorm-2022\venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
100%|██████████| 194/194 [00:12<00:00, 15.50it/s]


In [19]:
def __day_filter_values(data):
    output = {
        "ItemCode":data["input_data"]["ItemCode"]
    }

    __pred = data["week_prediction"]

    output["w1"] = __pred[0]
    output["w2"] = __pred[1]
    output["w3"] = __pred[2]
    output["w4"] = __pred[3]

    return output

def day_series_filter_out(model_data):
    output_df = {}
    for __key in tqdm(model_data.keys()):
        output_df[__key] = __day_filter_values(model_data[__key])

    output_df = pd.DataFrame.from_dict(output_df,orient='index')
    output_df = pd.melt(output_df,id_vars=["ItemCode"],value_vars=["w1","w2","w3","w4"])
    output_df.columns = ["ItemCode","Week","WeeklySales"]
    return output_df

In [20]:
day_series_filter_out(models_list)

100%|██████████| 194/194 [00:00<00:00, 38802.81it/s]


,ItemCode,Week,WeeklySales
0,3418,w1,54.165194
1,3427,w1,27.816553
2,7666,w1,41.537592
3,9925,w1,11.529598
4,16936,w1,62.034422
...,...,...,...
771,1101769,w4,35.539764
772,1103056,w4,12.494203
773,1105009,w4,6.564840
774,1105018,w4,5.854787
